# Imports

In [171]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pingouin as pg
from scipy import stats
import mne
import os
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif, RFECV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
from sklearn.decomposition import PCA
import shap
import statsmodels.api as sm
from statsmodels.formula.api import logit
import warnings


# Data Preparation

In [ ]:
np.random.seed(1000)

old = "old_results_all.csv"
young = "young_results_all.csv"
old_df = pd.read_csv(old)
young_df = pd.read_csv(young)

combined_df = pd.concat([old_df, young_df], ignore_index=True)

old_male_id = old_df[old_df['group'].str.contains('Male')]['subject_id'].unique()
old_female_id = old_df[old_df['group'].str.contains('Female')]['subject_id'].unique()

print(f"Number of old male participants: {len(old_male_id)}")
print(f"Number of old female participants: {len(old_female_id)}")


young_male_id = combined_df[(combined_df['age_group'] == 'young') & (combined_df['group'].str.contains('Male'))]['subject_id'].unique()
print(f"Number of young male participants: {len(young_male_id)}")

young_female_id = combined_df[(combined_df['age_group'] == 'young') & (combined_df['group'].str.contains('Female'))]['subject_id'].unique()
print(f"Number of young female participants: {len(young_female_id)}")

young_male_subjects = np.random.choice(young_male_id, size=18, replace=False)
young_female_subjects = np.random.choice(young_female_id, size=11, replace=False)

selected_young_subjects = np.concatenate([young_male_subjects, young_female_subjects])


# Create a new dataframe with the selected subjects both old and young
selected_subjects = np.concatenate([old_male_id, old_female_id, selected_young_subjects])
print("-" * 100)
print(f"Selected subjects: {selected_subjects}")
print(len(selected_subjects))

# Turn this into a dataframe
new_df = combined_df[combined_df['subject_id'].isin(selected_subjects)]

## Normalize the power


The following code implements a subject-specific normalization approach:

1. **Create a working copy** of the original dataframe to preserve raw data
2. **Iterate through each subject** to handle individual differences
3. **Process each condition-band combination** separately for proper normalization context
4. **Calculate normalization factor** as the sum of power values across all regions
5. **Normalize region-specific power** by dividing by the total power

This approach ensures that power comparisons between regions account for individual 
differences in overall signal strength while maintaining the relative distribution 
across brain regions.

In [ ]:
new_df_normalized = new_df.copy()
for subject in new_df['subject_id'].unique():
    subjectdf = new_df[new_df['subject_id'] == subject]

    for condition in subjectdf['condition'].unique():
        for band in subjectdf['band'].unique():
            normalizs = subjectdf[(subjectdf['condition'] == condition) & (subjectdf['band'] == band)]['power'].sum()
            for region in subjectdf['region'].unique():
                power = subjectdf[(subjectdf['condition'] == condition) & (subjectdf['band'] == band) & (subjectdf['region'] == region)]['power'].values[0]
                new_df_normalized.loc[(new_df_normalized['subject_id'] == subject) & (new_df_normalized['condition'] == condition) & (new_df_normalized['band'] == band) & (new_df_normalized['region'] == region),'normalized_power'] = power / normalizs

new_df_normalized


# Plots

In [ ]:
plt.figure(figsize=(14, 8))

# Create a new column that combines age_group and condition for grouping
new_df_normalized['group_condition'] = new_df_normalized['age_group'] + '_' + new_df_normalized['condition']

# Define a custom palette
palette = {
    'young_EC': '#2b6d89',    # Darker blue for young EC
    'young_EO': '#5dade2',    # Lighter blue for young EO
    'older_EC': '#e67e22',    # Darker orange for older EC
    'older_EO': '#f5b041'     # Lighter orange for older EO
}

# Create the grouped barplot
ax = sns.barplot(
    x='region', 
    y='normalized_power', 
    data=new_df_normalized, 
    hue='group_condition',
    palette=palette, 
    errorbar=('ci', 95),
    capsize=0.1,
    width=0.8,
    alpha=0.9
)

# Add grid and labels
plt.grid(axis='y', linestyle='--', alpha=0.7, zorder=0)
plt.title('Comparison of Normalized Power Between Age Groups and Conditions', fontsize=16, fontweight='bold', pad=15)
plt.xlabel('Brain Region', fontsize=12, fontweight='bold')
plt.ylabel('Normalized Power', fontsize=12, fontweight='bold')

# Format x-tick labels
plt.xticks(fontweight='bold', rotation=45, ha='right')

# Set y-axis limits
plt.ylim([0.10, 0.50])
plt.yticks(np.arange(0.10, 0.55, 0.05))

# Format the legend
plt.legend(
    title='Age Group - Condition', 
    title_fontsize=12, 
    frameon=True, 
    facecolor='white', 
    edgecolor='gray',
    bbox_to_anchor=(1.01, 0.5),
    loc='center left'
)

# Hide top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# Create an improved 2x2 facet grid
g = sns.catplot(
    data=new_df_normalized, 
    x='region',
    y='normalized_power',
    hue='band',
    col='age_group',
    row='condition',
    kind='point',
    dodge=True,
    errorbar=('ci', 95),
    palette=['#3b5998', '#20b2aa', '#66cdaa'],  # Better color contrast
    height=5.5,
    aspect=1.3,
    markers=['o', 's', 'D'],
    linestyles=['-', '--', '-.'],
    sharex=True,
    sharey=True,
    legend_out=True
)

# Enhance labels and titles
g.set_axis_labels("Brain Region", "Mean Normalized Power", fontsize=12)
g.set_titles("Age: {col_name} | Condition: {row_name}", fontsize=13, fontweight='bold')
g.set_xticklabels(fontsize=11, rotation=30, ha='right')  # Better readability

# Improve y-axis appearance
for ax in g.axes.flat:
    ax.set_ylim(0, 0.5)  # Consistent y-axis with some headroom
    ax.tick_params(axis='y', labelsize=11)
    ax.yaxis.grid(True, linestyle='--', alpha=0.7, color='#E0E0E0')
    
    # Add subtle background for better readability
    ax.set_facecolor('#f9f9f9')
    
    # Add subtle spines
    for spine in ax.spines.values():
        spine.set_color('#D0D0D0')
        spine.set_linewidth(0.8)


# Add a main title with better positioning
plt.suptitle('Normalized EEG Power by Region, Band, Age Group and Condition', 
             y=1.05, fontsize=16, fontweight='bold')
# After creating the plot, add these lines:
regions = new_df_normalized['region'].unique()
for i, ax in enumerate(g.axes.flat):
    ax.set_xticks(range(len(regions)))
    ax.set_xticklabels(regions, fontsize=11, rotation=30, ha='right')
# Adjust layout
plt.tight_layout(pad=2.0)
plt.subplots_adjust(top=0.9)

In [ ]:
# Create a 2x2 grid for our four conditions (age_group × condition)
fig, axes = plt.subplots(2, 2, figsize=(14, 12), sharex=True, sharey=True)

# Define age groups and conditions
age_groups = ['older', 'young']
conditions = ['EC', 'EO']

# Get unique region names to ensure consistency
regions = new_df_normalized['region'].unique()

# Custom colormap with better perceptual properties
cmap = sns.color_palette("viridis", as_cmap=True)

# Loop through each combination and create a heatmap
for i, condition in enumerate(conditions):
    for j, age in enumerate(age_groups):
        # Filter data for this subplot
        subset = new_df_normalized[(new_df_normalized['age_group'] == age) & 
                                   (new_df_normalized['condition'] == condition)]
        
        # Create pivot table for this specific combination
        pivot = subset.pivot_table(
            index='region', 
            columns='band', 
            values='normalized_power',
            aggfunc='mean'
        )
        
        # Ensure pivot table has consistent row order matching regions list
        pivot = pivot.reindex(regions)
        
        sns.heatmap(
            pivot, 
            ax=axes[i, j],
            cmap=cmap,
            vmin=0.10,
            vmax=0.45,
            annot=True,
            fmt='.2f',
            linewidths=0.5,
            cbar=False  # No colorbar on any subplot
        )
        
        # Add titles
        axes[i, j].set_title(f"Age: {age} | Condition: {condition}", 
                             fontsize=13, fontweight='bold')
        
        # Ensure region names are clearly displayed
        axes[i, j].set_yticklabels(regions, fontsize=11, fontweight='bold')
        
        # Set x/y labels only for outer subplots
        if i == 1:
            axes[i, j].set_xlabel('Frequency Band', fontsize=12)
        if j == 0:
            axes[i, j].set_ylabel('Brain Region', fontsize=12)

# Add a main title
plt.suptitle('EEG Power Matrix by Region, Band, Age Group and Condition', 
             y=0.98, fontsize=16, fontweight='bold')

# Add a single colorbar for the entire figure
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [x, y, width, height]
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(0.15, 0.45))
sm.set_array([])
cbar = fig.colorbar(sm, cax=cbar_ax)
cbar.set_label('Mean Normalized Power', fontsize=12)

# Adjust spacing
plt.tight_layout(rect=[0, 0, 0.9, 0.95])  # Make room for colorbar
plt.subplots_adjust(top=0.92)

In [ ]:
# 1. Two-way mixed ANOVA for each brain region
# Analyze differences between age groups, conditions, and their interaction

import pingouin as pg
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Function to perform two-way mixed ANOVA for each region
def region_anova(data, region_name):
    # Filter data for the specific region
    region_data = data[data['region'] == region_name].copy()
    
    # Reshape data for ANOVA
    aov = pg.mixed_anova(
        dv='normalized_power',         # Dependent variable
        within='condition',            # Within-subjects factor
        between='age_group',           # Between-subjects factor
        subject='subject_id',          # Subject identifier
        data=region_data
    )
    
    # Add region information to the results
    aov['brain_region'] = region_name
    
    return aov

# Run ANOVA for each region
regions = new_df_normalized['region'].unique()
anova_results = pd.DataFrame()

for region in regions:
    result = region_anova(new_df_normalized, region)
    anova_results = pd.concat([anova_results, result], ignore_index=True)

# Display ANOVA results with formatted p-values and interpretations
anova_summary = anova_results.copy()
anova_summary['p-value'] = anova_summary['p-unc'].apply(lambda p: f"{p:.4f}" + ("*" if p < 0.05 else ""))
anova_summary['significance'] = anova_summary['p-unc'].apply(lambda p: "Significant" if p < 0.05 else "Not significant")
anova_summary = anova_summary[['brain_region', 'Source', 'F', 'p-value', 'significance']]

print("Two-way mixed ANOVA results for each brain region:")
print(anova_summary)
print("\n* = p < 0.05")

# 2. Post-hoc tests for significant interactions or main effects
def posthoc_tests(data):
    posthoc_results = pd.DataFrame()
    regions = data['region'].unique()
    
    for region in regions:
        region_data = data[data['region'] == region].copy()
        
        # Post-hoc for age group differences (within each condition)
        for condition in ['EC', 'EO']:
            condition_data = region_data[region_data['condition'] == condition]
            
            # Compare age groups within condition
            ttest = pg.ttest(
                condition_data[condition_data['age_group'] == 'young']['normalized_power'],
                condition_data[condition_data['age_group'] == 'older']['normalized_power'],
                paired=False
            )
            
            ttest['brain_region'] = region
            ttest['comparison'] = f"young vs older ({condition})"
            posthoc_results = pd.concat([posthoc_results, ttest], ignore_index=True)
        
        # Post-hoc for condition differences (within each age group)
        for age in ['young', 'older']:
            age_data = region_data[region_data['age_group'] == age]
            
            # Compare conditions within age group
            ttest = pg.ttest(
                age_data[age_data['condition'] == 'EC']['normalized_power'],
                age_data[age_data['condition'] == 'EO']['normalized_power'],
                paired=True  # Paired because same subjects did both conditions
            )
            
            ttest['brain_region'] = region
            ttest['comparison'] = f"EC vs EO ({age})"
            posthoc_results = pd.concat([posthoc_results, ttest], ignore_index=True)
    
    return posthoc_results

# Run post-hoc tests
posthoc_results = posthoc_tests(new_df_normalized)

# Format post-hoc results for display
posthoc_summary = posthoc_results.copy()
posthoc_summary['p-value'] = posthoc_summary['p-val'].apply(lambda p: f"{p:.4f}" + ("*" if p < 0.05 else ""))
posthoc_summary['cohen-d'] = posthoc_summary['cohen-d'].apply(lambda d: f"{d:.3f}")
posthoc_summary = posthoc_summary[['brain_region', 'comparison', 'cohen-d', 'p-value']]

print("\nPost-hoc t-tests for significant effects:")
print(posthoc_summary)
print("\n* = p < 0.05")

# 3. Frequency band analysis with mixed ANOVA for each band
def band_anova(data, band_name):
    # Filter data for specific band
    band_data = data[data['band'] == band_name].copy()
    
    # Perform ANOVA
    aov = pg.mixed_anova(
        dv='normalized_power',
        within='condition',
        between='age_group',
        subject='subject_id',
        data=band_data
    )
    
    aov['frequency_band'] = band_name
    return aov

# Run ANOVA for each frequency band
bands = new_df_normalized['band'].unique()
band_anova_results = pd.DataFrame()

for band in bands:
    result = band_anova(new_df_normalized, band)
    band_anova_results = pd.concat([band_anova_results, result], ignore_index=True)

# Format band ANOVA results
band_summary = band_anova_results.copy()
band_summary['p-value'] = band_summary['p-unc'].apply(lambda p: f"{p:.4f}" + ("*" if p < 0.05 else ""))
band_summary['significance'] = band_summary['p-unc'].apply(lambda p: "Significant" if p < 0.05 else "Not significant")
band_summary = band_summary[['frequency_band', 'Source', 'F', 'p-value', 'significance']]

print("\nTwo-way mixed ANOVA results for each frequency band:")
print(band_summary)
print("\n* = p < 0.05")

# 4. Three-way mixed ANOVA (region × age group × condition)
# This tests if the pattern of age and condition effects differs across regions
def three_way_anova(data):
    # Create a long-format DataFrame suitable for three-way ANOVA
    # We'll need to reshape the data to have 'region' as a within-subject factor
    subjects = data['subject_id'].unique()
    regions = data['region'].unique()
    long_data = []
    
    for subject in subjects:
        subject_data = data[data['subject_id'] == subject]
        for condition in ['EC', 'EO']:
            condition_data = subject_data[subject_data['condition'] == condition]
            if len(condition_data) < len(regions):  # Skip if data incomplete
                continue
                
            age_group = condition_data['age_group'].iloc[0]
            
            for region in regions:
                region_data = condition_data[condition_data['region'] == region]
                if len(region_data) == 0:
                    continue
                    
                for band in bands:
                    band_data = region_data[region_data['band'] == band]
                    if len(band_data) == 0:
                        continue
                        
                    power = band_data['normalized_power'].iloc[0]
                    
                    long_data.append({
                        'subject_id': subject,
                        'age_group': age_group,
                        'condition': condition,
                        'region': region,
                        'band': band,
                        'normalized_power': power
                    })
    
    long_df = pd.DataFrame(long_data)
    
    # Perform three-way mixed ANOVA for each frequency band
    three_way_results = pd.DataFrame()
    
    for band in bands:
        band_data = long_df[long_df['band'] == band].copy()
        
        # Perform ANOVA with region as within-subject factor
        try:
            aov = pg.mixed_anova(
                dv='normalized_power',
                within=['condition', 'region'],  # Two within-subject factors
                between='age_group',             # Between-subjects factor
                subject='subject_id',
                data=band_data
            )
            
            aov['frequency_band'] = band
            three_way_results = pd.concat([three_way_results, aov], ignore_index=True)
        except:
            print(f"Could not perform full three-way ANOVA for {band} band.")
            # Use simplified approach with interaction terms
            simplified_aov = pg.rm_anova(
                dv='normalized_power',
                within=['condition', 'region'],
                subject='subject_id',
                data=band_data,
                detailed=True
            )
            simplified_aov['frequency_band'] = band
            three_way_results = pd.concat([three_way_results, simplified_aov], ignore_index=True)
    
    return three_way_results

# Run three-way ANOVA
try:
    three_way_results = three_way_anova(new_df_normalized)
    
    # Format results
    three_way_summary = three_way_results.copy()
    three_way_summary['p-value'] = three_way_summary['p-unc'].apply(lambda p: f"{p:.4f}" + ("*" if p < 0.05 else ""))
    three_way_summary['significance'] = three_way_summary['p-unc'].apply(lambda p: "Significant" if p < 0.05 else "Not significant")
    three_way_summary = three_way_summary[['frequency_band', 'Source', 'F', 'p-value', 'significance']]
    
    print("\nThree-way mixed ANOVA (region × age group × condition):")
    print(three_way_summary)
except Exception as e:
    print("\nCould not perform complete three-way ANOVA, using alternative approach...")
    
    # Alternative: region-wise test of condition × age_group interaction
    region_interaction_results = pd.DataFrame()
    
    for region in regions:
        for band in bands:
            region_band_data = new_df_normalized[(new_df_normalized['region'] == region) & 
                                                (new_df_normalized['band'] == band)]
            
            if len(region_band_data) > 0:
                model = pg.anova(
                    dv='normalized_power',
                    between=['age_group', 'condition'],
                    data=region_band_data,
                    detailed=True
                )
                
                model['region'] = region
                model['band'] = band
                region_interaction_results = pd.concat([region_interaction_results, model], ignore_index=True)
    
    # Show interaction results
    interaction_summary = region_interaction_results[region_interaction_results['Source'] == 'age_group * condition'].copy()
    interaction_summary['p-value'] = interaction_summary['p-unc'].apply(lambda p: f"{p:.4f}" + ("*" if p < 0.05 else ""))
    interaction_summary = interaction_summary[['region', 'band', 'F', 'p-value']]
    
    print("\nAge Group × Condition interaction for each region and band:")
    print(interaction_summary)

# 5. Effect size analysis for age differences by region
def calculate_effect_sizes(data):
    effect_sizes = pd.DataFrame()
    
    for region in regions:
        region_data = data[data['region'] == region]
        
        for band in bands:
            band_data = region_data[region_data['band'] == band]
            
            for condition in ['EC', 'EO']:
                condition_data = band_data[band_data['condition'] == condition]
                
                young_data = condition_data[condition_data['age_group'] == 'young']['normalized_power']
                older_data = condition_data[condition_data['age_group'] == 'older']['normalized_power']
                
                if len(young_data) > 0 and len(older_data) > 0:
                    # Calculate Cohen's d effect size
                    effect_size = pg.compute_effsize(
                        young_data, 
                        older_data,
                        paired=False,
                        eftype='cohen'
                    )
                    
                    effect_sizes = pd.concat([effect_sizes, pd.DataFrame({
                        'region': [region],
                        'band': [band],
                        'condition': [condition],
                        'effect_size': [effect_size],
                        'magnitude': [interpret_effect_size(effect_size)]
                    })], ignore_index=True)
    
    return effect_sizes

def interpret_effect_size(d):
    """Interpret Cohen's d effect size magnitude"""
    if abs(d) < 0.2:
        return "Negligible"
    elif abs(d) < 0.5:
        return "Small"
    elif abs(d) < 0.8:
        return "Medium" 
    else:
        return "Large"

# Calculate effect sizes
effect_sizes = calculate_effect_sizes(new_df_normalized)

# Format effect size results
effect_sizes['effect_size'] = effect_sizes['effect_size'].apply(lambda d: f"{d:.3f}")

print("\nEffect sizes (Cohen's d) for age group differences by region, band and condition:")
print(effect_sizes[['region', 'band', 'condition', 'effect_size', 'magnitude']])

# 6. Correlation analysis between regions for each age group and condition
def correlation_analysis(data):
    corr_results = []
    
    for age in ['young', 'older']:
        for condition in ['EC', 'EO']:
            # Filter data for this age group and condition
            filtered_data = data[(data['age_group'] == age) & 
                               (data['condition'] == condition)]
            
            # Calculate correlations between regions for each band
            for band in bands:
                band_data = filtered_data[filtered_data['band'] == band]
                
                # Create pivot table with regions as columns
                pivot_data = band_data.pivot_table(
                    index='subject_id',
                    columns='region',
                    values='normalized_power',
                    aggfunc='mean'
                )
                
                # Calculate correlation matrix
                correlation_matrix = pivot_data.corr(method='pearson')
                
                # Extract non-redundant correlations (upper triangle)
                for i, region1 in enumerate(regions[:-1]):
                    for region2 in regions[i+1:]:
                        corr_coef = correlation_matrix.loc[region1, region2]
                        
                        # Calculate significance
                        r, p_value = stats.pearsonr(
                            pivot_data[region1].dropna(),
                            pivot_data[region2].dropna()
                        )
                        
                        corr_results.append({
                            'age_group': age,
                            'condition': condition,
                            'band': band,
                            'region1': region1,
                            'region2': region2,
                            'correlation': corr_coef,
                            'p_value': p_value
                        })
    
    return pd.DataFrame(corr_results)

# Run correlation analysis
correlation_results = correlation_analysis(new_df_normalized)

# Format correlation results
correlation_summary = correlation_results.copy()
correlation_summary['correlation'] = correlation_summary['correlation'].apply(lambda r: f"{r:.3f}")
correlation_summary['p-value'] = correlation_summary['p_value'].apply(lambda p: f"{p:.4f}" + ("*" if p < 0.05 else ""))
correlation_summary = correlation_summary[['age_group', 'condition', 'band', 'region1', 'region2', 'correlation', 'p-value']]

print("\nCorrelations between brain regions for each age group and condition:")
# Show only significant correlations
significant_corrs = correlation_summary[correlation_summary['p-value'].str.contains('\*')]
print(significant_corrs)
print("\n* = p < 0.05")

# Statistical Analysis

## Converting dataset **Long** -> **Wide**

Ok So given the current format of my data set I will be turning my dataframe from long to wide format

In [ ]:
new_df_normalized['feature_name'] = new_df_normalized['condition'] + '_' + new_df_normalized['band'] + '_' + new_df_normalized['region'] # a new column that contains the feature name
print(new_df_normalized['feature_name'])

In [179]:
new_df_normalized
new_df_normalized = new_df_normalized.drop('power', axis=1)

In [ ]:
new_df_normalized

In [ ]:
wide_df = new_df_normalized.pivot_table(
    index = "subject_id", # unique identifier for each subject
    columns= 'feature_name', # the feature name
    values = 'normalized_power', # the values to be pivoted
    aggfunc = 'first' # how to handle duplicate values
)
wide_df = wide_df.reset_index()
wide_df.head(5)


### Adding Demographics


In [ ]:
age_group = new_df_normalized[['subject_id', 'age_group']].drop_duplicates()
value_counts = age_group['age_group'].value_counts()
print(f"Age group info: ")
print(value_counts)
gender_info = new_df_normalized[['subject_id', 'group']].drop_duplicates()
print(f"Gender info: ")
gender_info['gender'] = gender_info['group'].apply(
    lambda x: 'Female' if 'Female' in x else 'Male')
gender_value_counts = gender_info['gender'].value_counts()
print(gender_value_counts)

wide_df = wide_df.merge(age_group, on='subject_id')
wide_df = wide_df.merge(gender_info[['subject_id', 'gender']], on='subject_id')

wide_df.head(5)

# List the column names

In [ ]:
wide_df.shape

for col in wide_df.columns:
    print(col)






## Random Forest

### Building the model

In [ ]:
import optuna
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score
X = wide_df.drop(columns=['subject_id', 'age_group', 'gender'])
y = wide_df['age_group']
y_binary = (y == 'older').astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_binary, test_size=0.25, random_state=42, stratify=y_binary
)

print(f"\nTraining set: {X_train.shape[0]} subjects")
print(f"Testing set: {X_test.shape[0]} subjects")


def objective(trial):
    # Define the hyperparameter search space
    k_best = trial.suggest_int('k_best', 10, min(50, X_train.shape[1]))
    
    # Random Forest hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    
    # Create pipeline with trial parameters
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=k_best)),
        ('classifier', RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        ))
    ])
    
    # Use cross-validation to evaluate the model
    scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='roc_auc')
    return scores.mean()

# Create and run the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, n_jobs=-1)  # Adjust number of trials as needed

# Print the best parameters and score
print("Best trial:")
trial = study.best_trial
print(f"  ROC AUC: {trial.value:.4f}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Create final model with best parameters
best_k = trial.params['k_best']
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(f_classif, k=best_k)),
    ('classifier', RandomForestClassifier(
        n_estimators=trial.params['n_estimators'],
        max_depth=trial.params['max_depth'],
        min_samples_split=trial.params['min_samples_split'],
        min_samples_leaf=trial.params['min_samples_leaf'],
        random_state=42
    ))
])

# Train on the full training set
pipeline.fit(X_train, y_train)

# Evaluate on the test set
y_pred = pipeline.predict(X_test)
test_accuracy = accuracy_score(y_test, y_pred)
test_roc_auc = roc_auc_score(y_test, y_pred)

print(f"\nTest accuracy: {test_accuracy:.4f}")
print(f"Test ROC AUC: {test_roc_auc:.4f}")

# To see which features were selected
selected_features = X.columns[pipeline.named_steps['selector'].get_support()]
print(f"\nSelected features ({len(selected_features)}):")
print(selected_features.tolist())

### Explanation of the Metrics Above

**Accuracy** How many times the model correctly classified of all samples

**F1** This is the harmonic mean between the precision and recall


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve
y_pred = pipeline.predict(X_test)
print("\nModel Performance:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.3f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_pred):.3f}")

Technically a 88 percent performance on 5 fold could be very accaptable but I dont know for a sample size as small as this one  will need to read more about this 

In [ ]:
# 6. Cross-validation results
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipeline, X, y_binary, cv=cv, scoring='accuracy')
print(f"\nCross-validation accuracy: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
# Visualize cross-validation performance
plt.figure(figsize=(10, 6))
plt.bar(range(1, 6), cv_scores, color='skyblue', edgecolor='black')
plt.axhline(cv_scores.mean(), color='red', linestyle='--', label=f'Mean: {cv_scores.mean():.3f}')
plt.xlabel('Fold', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy', fontsize=12, fontweight='bold')
plt.title('5-Fold Cross-Validation Performance', fontsize=14, fontweight='bold')
plt.xticks(range(1, 6))
plt.ylim(0, 1)
plt.legend()
plt.tight_layout()


In [ ]:
selected_indices = pipeline.named_steps['selector'].get_support(indices=True)
selected_features = X.columns[selected_indices].tolist()
importances = pipeline.named_steps['classifier'].feature_importances_
feature_importances = pd.DataFrame({
    'Feature': selected_features,
    'Importance': importances
})
feature_importances = feature_importances.sort_values('Importance', ascending=False)
print("Top 10 most important features:")
print(feature_importances.head(10))

# Get feature importance from Random Forest
selected_X_train = pipeline.named_steps['selector'].transform(X_train)
rf_classifier = pipeline.named_steps['classifier']
# Map importances back to original feature names
importances = pd.Series(
    rf_classifier.feature_importances_,
    index=selected_features
)

# Sort and visualize top features
plt.figure(figsize=(10, 8))
importances = importances.sort_values(ascending=False)
ax = importances[:15].plot(kind='barh')
ax.invert_yaxis()  # Display with highest importance at the top
plt.title('Top 15 Features Importance for Age Group Prediction')
plt.tight_layout()



In [ ]:
# Enhance the statistical testing function
def add_enhanced_statistical_testing(features, df):
    results = []
    
    for feature in features:
        parts = feature.split('_')
        condition = parts[0]
        band = parts[1]
        region = '_'.join(parts[2:])
        
        # Get values for each group
        young_values = df[(df['age_group'] == 'young') & 
                         (df['condition'] == condition) & 
                         (df['band'] == band) & 
                         (df['region'] == region)]['normalized_power']
        
        older_values = df[(df['age_group'] == 'older') & 
                         (df['condition'] == condition) & 
                         (df['band'] == band) & 
                         (df['region'] == region)]['normalized_power']
        
        # Calculate means for both groups
        young_mean = young_values.mean()
        older_mean = older_values.mean()
        
        # Calculate percent difference
        percent_diff = ((older_mean - young_mean) / young_mean) * 100
        
        # Run t-test
        t_stat, p_value = stats.ttest_ind(older_values, young_values, equal_var=False)
        
        # Determine which group has higher power
        direction = "Older > Young" if t_stat > 0 else "Young > Older"
        
        # Add to results
        results.append({
            'Feature': feature,
            'Condition': condition,
            'Band': band,
            'Region': region,
            'Young Mean': young_mean,
            'Older Mean': older_mean,
            'Percent Diff': percent_diff,
            't-statistic': t_stat,
            'p-value': p_value,
            'Direction': direction,
            'Significant': p_value < 0.05,
            'Significance': '***' if p_value < 0.001 else '**' if p_value < 0.01 else '*' if p_value < 0.05 else 'ns'
        })
    
    return pd.DataFrame(results)

# Run enhanced statistical testing
enhanced_stats = add_enhanced_statistical_testing(feature_importances['Feature'], new_df_normalized)

# Group by region and sort by absolute t-statistic for better interpretation
enhanced_stats['Abs_t'] = abs(enhanced_stats['t-statistic'])
enhanced_stats_sorted = enhanced_stats.sort_values(['Region', 'Abs_t'], ascending=[True, False])

# Format p-values
enhanced_stats_sorted['p-value_fmt'] = enhanced_stats_sorted['p-value'].apply(
    lambda x: f"{x:.3e}" if x < 0.001 else f"{x:.3f}")

# Create a more readable display function
def display_region_stats(stats_df):
    # Define ANSI color codes for terminal output
    BOLD = '\033[1m'
    GREEN = '\033[92m'
    RED = '\033[91m'
    BLUE = '\033[94m'
    END = '\033[0m'
    
    # List of regions in the order we want to display (posterior to anterior, following PASA)
    regions = ['occipital', 'parietal', 'temporal', 'frontal']
    
    # Print header
    print(f"\n{BOLD}Statistical Analysis of EEG Power Differences Between Age Groups{END}")
    for region in regions:
        region_data = stats_df[stats_df['Region'] == region].copy()
        if len(region_data) > 0:
            # Print region header with posterior/anterior classification
            region_type = "POSTERIOR" if region in ['occipital', 'parietal'] else "ANTERIOR"
            print(f"\n{BOLD}{BLUE}Region: {region.upper()} ({region_type}){END}")
            print("-" * 100)
            
            # Print column headers
            print(f"{BOLD}{'Feature':<20} {'Band':<10} {'Condition':<10} {'Young':<10} {'Older':<10} {'%Diff':<10} {'t-stat':<10} {'p-value':<15} {'Significance':<15}{END}")
            print("-" * 100)
            
            # Print each row
            for _, row in region_data.iterrows():
                # Color code based on direction (green for older>young, red for young>older)
                color = GREEN if row['Direction'] == 'Older > Young' else RED
                sign = row['Significance']
                
                print(f"{row['Feature']:<20} {row['Band']:<10} {row['Condition']:<10} "
                      f"{row['Young Mean']:.3f}    {row['Older Mean']:.3f}    "
                      f"{color}{row['Percent Diff']:+.1f}%{END}    "
                      f"{row['t-statistic']:+.2f}    {row['p-value_fmt']:<15} "
                      f"{color}{sign}{END}")
    
    # Print summary relevant to the PASA model
    print(f"\n{BOLD}Summary of Findings Relevant to PASA Model:{END}")
    posterior_regions = stats_df[stats_df['Region'].isin(['occipital', 'parietal'])]
    anterior_regions = stats_df[stats_df['Region'].isin(['temporal', 'frontal'])]
    
    posterior_direction = posterior_regions['t-statistic'].mean() < 0
    anterior_direction = anterior_regions['t-statistic'].mean() > 0
    
    pasa_support = posterior_direction and anterior_direction
    
    print(f"Posterior regions (occipital, parietal): "
          f"{'Young > Older' if posterior_direction else 'Older > Young'} "
          f"(Mean t-stat: {posterior_regions['t-statistic'].mean():.2f})")
    
    print(f"Anterior regions (temporal, frontal): "
          f"{'Older > Young' if anterior_direction else 'Young > Older'} "
          f"(Mean t-stat: {anterior_regions['t-statistic'].mean():.2f})")
    

# Display the enhanced statistics
display_region_stats(enhanced_stats_sorted)

In [ ]:
# Region-wise feature importance analysis
region_importances = {}

for feature, importance in zip(selected_features, rf_classifier.feature_importances_):
    # Extract region from feature name (assuming format like 'EC_alpha_occipital')
    region = feature.split('_')[-1]
    
    if region not in region_importances:
        region_importances[region] = 0
    region_importances[region] += importance

# Plot region-wise importance
plt.figure(figsize=(10, 6))
regions = list(region_importances.keys())
importances = list(region_importances.values())
colors = ['#3498db', '#e67e22', '#2ecc71', '#e74c3c']  # Same colors as before

plt.bar(regions, importances, color=colors, edgecolor='black')
plt.xlabel('Brain Region', fontsize=12, fontweight='bold')
plt.ylabel('Cumulative Importance', fontsize=12, fontweight='bold')
plt.title('Brain Region Contribution to Age Classification', fontsize=14, fontweight='bold')
plt.xticks(rotation=0, fontweight='bold')
plt.grid(axis='y', linestyle='--', alpha=0.3)
plt.tight_layout()